# Horovod-Infiniband-Benchmark


## Introduction

This recipe shows how to reproduce [Horovod distributed training benchmarks](https://github.com/uber/horovod/blob/master/docs/benchmarks.md) using Azure Batch AI.

Currently Batch AI has no native support for Horovod framework, but it's easy to run it using customtoolkit and job preparation command line.


## Details

- Official Horovod Benchmark [scripts](https://github.com/alsrgv/benchmarks/tree/master/scripts/tf_cnn_benchmarks) will be used;
- The job will be run on standard tensorflow container ```tensorflow/tensorflow:1.4.0-gpu```;
- Horovod framework and IntelMPI will be installed in the container using job preparation command line. Note, you can build your own docker image containing tensorflow and horovod instead.
- Benchmark scripts will be downloaded to GPU nodes using job preparation command line as well.
- This sample needs to use at lesat two `STANDARD_NC24r` nodes, please be sure you have enough quota
- Standard output of the job will be stored on Azure File Share.

## Instructions

### Install Dependencies and Create Configuration file.
Follow [instructions](/recipes) to install all dependencies and create configuration file.

### Read Configuration and Create Batch AI client

In [ ]:
from __future__ import print_function

import time
from datetime import datetime
import os
import sys
import zipfile

from azure.storage.file import FileService, FilePermissions
import azure.mgmt.batchai.models as models

# utilities.py contains helper functions used by different notebooks
sys.path.append('../../')
import utilities

cfg = utilities.Configuration('../../configuration.json')
client = utilities.create_batchai_client(cfg)

### Create File Share

For this example we will create a new File Share with name `batchaisample` under your storage account. This share will be populated with sample scripts and will contain job's output.

**Note** You don't need to create new file share for every cluster. We are doing this in this sample to simplify resource management for you.

In [ ]:
azure_file_share_name = 'batchaisample'
service = FileService(cfg.storage_account_name, cfg.storage_account_key)
service.create_share(azure_file_share_name, fail_on_exist=False)

## Configure Compute Cluster

- For this example we will use a gpu cluster of `STANDARD_NC24r` nodes, which equip with infiniband device. Number of nodes in the cluster is configured with `nodes_count` variable, and 2 nodes will be used by default;
- Please be sure you have enough core quota to create at lesat two `STANDARD_NC24r` nodes
- We need to use the latest `UbuntuServer 16.04-LTS` as the host image, which is compatible with infiniband.
- We will mount file share at folder with name `external`. Full path of this folder on a computer node will be `$AZ_BATCHAI_MOUNT_ROOT/external`;
- We will call the cluster `nc24r`.


So, the cluster will have the following parameters:

In [ ]:
azure_file_share = 'external'
nodes_count = 2
cluster_name = 'nc24r'

volumes = models.MountVolumes(
    azure_file_shares=[
        models.AzureFileShareReference(
            account_name=cfg.storage_account_name,
            credentials=models.AzureStorageCredentialsInfo(
                account_key=cfg.storage_account_key),
            azure_file_url = 'https://{0}.file.core.windows.net/{1}'.format(
                cfg.storage_account_name, azure_file_share_name),
            relative_mount_path=azure_file_share)
    ]
)

parameters = models.ClusterCreateParameters(
    location=cfg.location,
    vm_size="STANDARD_NC24r",
    virtual_machine_configuration=models.VirtualMachineConfiguration(
        image_reference=models.ImageReference(
            publisher="Canonical",
            offer="UbuntuServer",
            sku="16.04-LTS",
            version="latest")),
    scale_settings=models.ScaleSettings(
        manual=models.ManualScaleSettings(target_node_count=nodes_count)
    ),
    node_setup=models.NodeSetup(
        mount_volumes=volumes
    ),
    user_account_settings=models.UserAccountSettings(
        admin_user_name=cfg.admin,
        admin_user_password=cfg.admin_password,
        admin_user_ssh_public_key=cfg.admin_ssh_key
    )
)

### Create Compute Cluster

In [ ]:
_ = client.clusters.create(cfg.resource_group, cluster_name, parameters).result()

### Monitor Cluster Creation

utilities.py contains a helper function allowing to wait for the cluster to become available - all nodes are allocated and finished preparation.

In [ ]:
cluster = client.clusters.get(cfg.resource_group, cluster_name)
utilities.print_cluster_status(cluster)

### Deploy Job Preparation Script and Configure the Input Directories

Create a folder in the file share and upload the sample script to it.

In [ ]:
samples_dir = "horovod_samples"
service = FileService(cfg.storage_account_name, cfg.storage_account_key)
service.create_directory(
    azure_file_share_name, samples_dir, fail_on_exist=False)

Upload the job preparation script, that does the following tasks:
- Install essential packages for infiniband support
- Download benchmark sample
- Install IntelMPI binary
- Install honovod framework

In [ ]:
service.create_file_from_path(
    azure_file_share_name, samples_dir, 'jobprep_benchmark.sh', 'jobprep_benchmark.sh')

- The job needs to know where to find train_mnist.py script (the chainer will download MNIST dataset on its own). So, we will configure an input directory for the script:

In [ ]:
input_directories = [
    models.InputDirectory(
        id='SCRIPTS',
        path='$AZ_BATCHAI_MOUNT_ROOT/{0}/{1}'.format(azure_file_share, samples_dir))
]

The job will be able to reference those directories using ```$AZ_BATCHAI_INPUT_SCRIPT``` environment variable.

### Configure Output Directories
We will store standard and error output of the job in File Share:

In [ ]:
std_output_path_prefix = "$AZ_BATCHAI_MOUNT_ROOT/{0}".format(azure_file_share)

### Configure Job

- Will use configured previously input and output directories;
- We will use custom toolkit job to run tensorflow_mnist.py on multiple nodes (use node_count parameter to specify number of nodes). Note, Batch AI will create a host list for the job, it can be found via ```$AZ_BATCH_HOST_LIST``` environment variable;
- Horovod framework, IntelMPI and benchmark sample scripts will be installed by job preparation command line;
- Will output standard output and error streams to file share.
- If you are insterested using TCP instead, please replace ```-env I_MPI_FABRICS=dapl -hosts $AZ_BATCH_HOST_LIST -env I_MPI_DAPL_PROVIDER=ofa-v2-ib0 -env I_MPI_DYNAMIC_CONNECTION=0``` with ```-env I_MPI_FABRICS=tcp``` in the command line

In [ ]:
job_name = datetime.utcnow().strftime("hvdbenchmark_%m_%d_%Y_%H%M%S")
numWorkers = nodes_count * 4
parameters = models.job_create_parameters.JobCreateParameters(
     location=cfg.location,
     cluster=models.ResourceId(cluster.id),
     node_count=nodes_count,
     input_directories=input_directories,
     std_out_err_path_prefix=std_output_path_prefix,
     container_settings=models.ContainerSettings(
         models.ImageSourceRegistry(image='tensorflow/tensorflow:1.4.0-gpu')),
     job_preparation=models.JobPreparation(
         command_line="bash $AZ_BATCHAI_INPUT_SCRIPTS/jobprep_benchmark.sh"),
     custom_toolkit_settings = models.CustomToolkitSettings(
         command_line="source /opt/intel/compilers_and_libraries_2017.4.196/linux/mpi/intel64/bin/mpivars.sh; cd $AZ_BATCHAI_JOB_TEMP/benchmarks/; mpirun -n {0} -ppn 4 -env I_MPI_FABRICS=dapl -env I_MPI_DAPL_PROVIDER=ofa-v2-ib0 -env I_MPI_DYNAMIC_CONNECTION=0 python scripts/tf_cnn_benchmarks/tf_cnn_benchmarks.py --model resnet101 --batch_size 64 --variable_update horovod".format(str(numWorkers))))


### Create a training Job and wait for Job completion

- Wait for job to complete, and keep streaming the stdout log
- When the job completes, you will see the number of images processed per second by the end of the log

In [ ]:
_ = client.jobs.create(cfg.resource_group, job_name, parameters).result()
print('Created Job: {}'.format(job_name))

### Wait for Job to Finish
The job will start running when the cluster will have enought idle nodes. The following code waits for job to start running printing the cluster state. During job run, the code prints current content of stderr.txt.

**Note** Execution may take several minutes to complete.

In [ ]:
utilities.wait_for_job_completion(client, cfg.resource_group, job_name, cluster_name, 'stdouterr', 'stdout.txt')

### Download stdout.txt and stderr.txt files for the Job and job preparation command

In [ ]:
files = client.jobs.list_output_files(cfg.resource_group, job_name, models.JobsListOutputFilesOptions("stdouterr")) 
for file in list(files):
    utilities.download_file(file.download_url, file.name)
print("All files Downloaded")

### Delete the Job

In [ ]:
client.jobs.delete(cfg.resource_group, job_name)

### Delete the Cluster
When you are finished with the sample and don't want to submit any more jobs you can delete the cluster using the following code.

In [ ]:
client.clusters.delete(cfg.resource_group, cluster_name)

### Delete File Share
When you are finished with the sample and don't want to submit any more jobs you can delete the file share completely with all files using the following code.

In [ ]:
service = FileService(cfg.storage_account_name, cfg.storage_account_key)
service.delete_share(azure_file_share_name)